This notebook will be used for a coursera captone: Battle of the Neighborhoods

In [1]:
#install necessary modules
!pip install bs4
!pip install requests
!pip install html5lib

#import used libraries
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [2]:
#Based on week2 lab, saving this infor in cell for use later

CLIENT_ID = 'UATKCT2MMK3S1ZP11HH3BR0YJ42W1ELEZMEVXD0PLWGGAPFO' # your Foursquare ID
CLIENT_SECRET = 'HTVNXNE1GPEFQ4RJKPX3BIAR1BYG3WYQQ3KZSFUFF2MLBRHN' # your Foursquare Secret
ACCESS_TOKEN = 'M4LRXZBVTKQ3KORATCGE5SLAU4UXMZM0DTMSBPYINLG5BLVD' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30


In [3]:
#extract html of wikipedia page with toronto Postal Code Data
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
data  = requests.get(url).text
soup = BeautifulSoup(data,"html.parser")
tables = soup.find_all('table') 

#select table of interest. Based on wikipedia page observation, this will be the first table
table_index=0

In [4]:
#create df with columns as specified in the assignment
t_hoods = pd.DataFrame(columns=["PostalCode", "Borough", "Neighborhood"])


#iterate thorugh each row
for row in tables[table_index].tbody.find_all("tr"):
    
    
    #find all cells in the currently iterated row
    col = row.find_all("td")
    
    
    #if the cell is not blank then for every cell, append the contents to the dataframe in the 'PostalCode' column
    if (col != []):
        for i in range(0,len(col)):
            postalcode=col[i].text.strip()
            borough = ''
            neighborhood = ''
            t_hoods = t_hoods.append({"PostalCode":postalcode, "Borough":borough, "Neighborhood":neighborhood}, ignore_index=True)

            
#Populate the 'Borouhgs' column with 'PoistalCode' column values after stripping the first 3 letters
#(this is the postal code based on the data structure)          
t_hoods['Borough']=t_hoods['PostalCode'].str[3:]


#create a list from the 'Boroughs' column and iterate through the list,
#splitting the string on an opening parentheses and saving this into another list. 
#Then replace the Boroughs column with these updated list values, 
#these are the Boroughs separated from the neighborhoods and postal codes 
breakoff = list(t_hoods["Borough"]) 
bors=[]
for i in breakoff:
    bors.append(i.split('(')[0])
t_hoods['Borough']=bors


#from the unbroken 'PostalCodes' string, extract the text within parenthese. 
#based on the data structure these are the Neighborhoods 
t_hoods['Neighborhood']=t_hoods['PostalCode'].str.extract('.*\((.*)\).*')


#create a new df from the current, excluding any rows where the 'Borough' entry is 'Not assigned'
t_hoods=t_hoods[~t_hoods.Borough.str.contains('Not assigned')]


#create a list from the 'Neighborhood' column and iterate through the list,
#replace ' /' with ',' for formatting as directed by the assignment 
#Then replace the Neighborhood column with these updated list values
hood=[]
hoods=list(t_hoods['Neighborhood'])
for i in hoods:
    hood.append(i.replace(' /',','))
t_hoods['Neighborhood']=hood

#Strip all text except the first 3 characters from the 'PostalCodes' column
#this leaves the column as the postal codes based on the datastructure
t_hoods['PostalCode']=t_hoods['PostalCode'].str[0:3]

#reset index and display the first 12 rows
t_hoods.reset_index(drop=True).head(12)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [5]:
#display the size of the df as requested by the assignment
t_hoods.shape

(103, 3)